In [ ]:
import whisper
from transformers import pipeline
import subprocess
import os

# Carregar o modelo Whisper
modelo_whisper = whisper.load_model("large")

# Função para converter áudio para .wav usando ffmpeg
def converter_para_wav(arquivo_audio):
    arquivo_wav = "temp_audio.wav"
    comando_ffmpeg = ["ffmpeg", "-i", arquivo_audio, "-ar", "16000", "-ac", "1", "-y", arquivo_wav]
    subprocess.run(comando_ffmpeg, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return arquivo_wav

# Função para transcrever o áudio
def transcrever_audio(arquivo_audio):
    # Converte o arquivo de áudio para .wav, se necessário
    if not arquivo_audio.endswith(".wav"):
        arquivo_audio = converter_para_wav(arquivo_audio)
    resultado = modelo_whisper.transcribe(arquivo_audio)
    
    # Remove o arquivo temporário .wav se foi criado
    if "temp_audio.wav" in arquivo_audio:
        os.remove(arquivo_audio)
        
    return resultado['text']

# Função para resumir o texto
def resumir_texto(texto):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    resumo = summarizer(texto, max_length=150, min_length=30, do_sample=False)
    return resumo[0]['summary_text']

# Função principal para transcrever, resumir e salvar em um arquivo .txt
def processar_audio(arquivo_audio):
    transcricao = transcrever_audio(arquivo_audio)
    resumo = resumir_texto(transcricao)
    
    # Salvar a transcrição e o resumo em um arquivo .txt
    with open("output.txt", "w", encoding="utf-8") as arquivo:
        arquivo.write("Transcrição:\n")
        arquivo.write(transcricao + "\n\n")
        arquivo.write("Resumo:\n")
        arquivo.write(resumo)
    
    print("Transcrição e resumo salvos em 'resumo.txt'.")

# Exemplo de uso com um arquivo mp3
arquivo_audio = r"C:\Users\Gabriel\www\ExerciciosPython\audio_teste.opus"  # substitua pelo caminho do seu arquivo de áudio
processar_audio(arquivo_audio)


C:\Users\Gabriel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|█████████████████████████████████████| 2.88G/2.88G [15:16<00:00, 3.37MiB/s]
C:\Users\Gabriel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` wil

Transcrição:  Eu não me abro em longa, eu não estou em longa Os mais íntimos me chamam de pila, escuta só Eu curto com meu trans na baixa e escondido da minha namorada E isso não tem problema, porque ela conhece o esquema Não dou meu braço a você e dou o cu a roer Matheus Duarte, quem me introduziu a essa vida Já comi o neguinho na baixa e eles me chamam de pila De segunda a sexta, estou com a minha amada Mas no sábado eu gosto de dar uma bimbada com transitais Matheus Duarte, quem me introduziu a essa vida Já comi o neguinho na baixa e eles me chamam de pila De segunda a sexta, estou com a minha amada Mas no sábado eu gosto de dar uma bimbada De dar uma bimbada com transitais É tudo resenha, pilunga, não fique bravo Meu nome é Bilunga e eu gosto de comer viado Tudo resenha, pilunga, não fique bravo Meu nome é Bilunga e eu gosto de comer viado Eu curto com meu trans na baixa e escondido da minha namorada E isso não tem problema, porque ela conhece o esquema Não dou meu braço a você e d

In [2]:
import whisper
from openai import OpenAI
import subprocess
import os

# Carregar o modelo Whisper
modelo_whisper = whisper.load_model("large")

# Inicializar o cliente OpenAI
client = OpenAI() # OBS: Definir API_KEY nas variaveis de ambiente

# Função para converter áudio para .wav usando ffmpeg
def converter_para_wav(arquivo_audio):
    arquivo_wav = "temp_audio.wav"
    comando_ffmpeg = ["ffmpeg", "-i", arquivo_audio, "-ar", "16000", "-ac", "1", "-y", arquivo_wav]
    subprocess.run(comando_ffmpeg, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return arquivo_wav

# Função para transcrever o áudio
def transcrever_audio(arquivo_audio):
    # Converte o arquivo de áudio para .wav, se necessário
    if not arquivo_audio.endswith(".wav"):
        arquivo_audio = converter_para_wav(arquivo_audio)
    resultado = modelo_whisper.transcribe(arquivo_audio)
    return resultado['text']

# Função para gerar um relatório detalhado
def gerar_relatorio_detalhado(texto):
    prompt = f"Sabendo que se trata de um texto que sera encaminhado para uma secretaria municipal. Primeiro classifique o texto em algumas dessas opções: Meio Ambiente, Educação, Fazenda, Infraestrutura, Abastecimento pesca e agricultura, Turismo, Saúde, Esporte. Depois escreva uma subcategoria. Em seguida defina o tema principal abordado. Após esses dois passos, faça um resumo bem objetivo. Um exemplo hipotetico seria: Categoria: infraestrutura Subcategoria: praças Tema: iluminação Assunto principal: “puxar do texto”. De acordo com o que informei, realize o processo com o seguinte conteúdo: {texto}"
    
    # Fazendo a chamada à API da OpenAI
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    # Verificando se o retorno tem o formato esperado
    try:
        relatorio = response.choices[0].message.content
    except AttributeError:
        print("A resposta da API não está no formato esperado.")
        relatorio = "Erro ao gerar o relatório."
    return relatorio

# Função principal para transcrever e gerar relatório
def processar_audio(arquivo_audio):
    transcricao = transcrever_audio(arquivo_audio)
    relatorio = gerar_relatorio_detalhado(transcricao)
    
    # Salvar transcrição e relatório em um arquivo .txt
    with open("resultado_audio.txt", "w", encoding="utf-8") as arquivo:
        # arquivo.write("Transcrição:\n")
        # arquivo.write(transcricao + "\n\n")
        arquivo.write("Relatório Detalhado:\n")
        arquivo.write(relatorio)
        
    print("Processamento concluído. Confira 'resultado_audio.txt' para o relatório completo.")

# Exemplo de uso com um arquivo mp3
arquivo_audio = "C:\\Users\\Gabriel\\www\\AudioAI\\audio_teste2.opus"  # substitua pelo caminho do seu arquivo de áudio
processar_audio(arquivo_audio)


C:\Users\Gabriel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues r

Processamento concluído. Confira 'resultado_audio.txt' para o relatório completo.
